####Q5

Use a Rashpberry Pi based data aquistion system and use that to measure the natural frequencies of the 2-DOF rotational plus translational system in the applied mechanics lab using FFTs methods.

sample Python code that can be used to measure the natural frequencies of a 2-DOF rotational plus translational system in the applied mechanics lab using a Raspberry Pi based data acquisition system and FFT methods. 
python

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

# Import required libraries for data acquisition and FFT analysis
import RPi.GPIO as GPIO
import spidev
import math

# Set up SPI interface for data acquisition from sensor
spi = spidev.SpiDev()
spi.open(0,0)
spi.max_speed_hz = 1350000

# Define GPIO pin for data ready signal from sensor
DRDY = 26

# Set up GPIO pin for data ready signal from sensor
GPIO.setmode(GPIO.BCM)
GPIO.setup(DRDY, GPIO.IN)

# Define sampling frequency and time duration for data acquisition
fs = 5000 # Hz
duration = 5 # seconds
num_samples = fs * duration

# Define frequency range of interest for FFT analysis
f_min = 0 # Hz
f_max = 500 # Hz
num_freqs = 500

# Initialize data buffer for sensor readings
data_buf = []

# Wait for data ready signal from sensor
while not GPIO.input(DRDY):
    time.sleep(0.001)

# Start data acquisition
for i in range(num_samples):
    # Read data from sensor
    adc_value = spi.xfer2([0x06,0,0])
    adc_value = ((adc_value[1]&0x0F)<<8)+adc_value[2]
    voltage = adc_value*3.3/4096
    
    # Append data to buffer
    data_buf.append(voltage)
    
    # Wait for next sample
    time.sleep(1/fs)

# Convert data buffer to numpy array for FFT analysis
data = np.array(data_buf)

# Compute FFT of data using numpy library
fft_data = np.fft.fft(data)
freqs = np.fft.fftfreq(num_samples, 1/fs)
idx = np.where((freqs > f_min) & (freqs < f_max))
freqs = freqs[idx]
fft_data = fft_data[idx]
abs_fft_data = np.abs(fft_data)/num_samples

# Find peaks in frequency spectrum for natural frequency detection
peaks, _ = scipy.signal.find_peaks(abs_fft_data, height=0.1)

# Plot frequency spectrum with peaks
plt.plot(freqs, abs_fft_data)
plt.plot(freqs[peaks], abs_fft_data[peaks], 'x')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.title('Frequency Spectrum')
plt.show()

# Determine natural frequencies of the system from peak locations
natural_freqs = freqs[peaks]

print('Natural Frequencies:', natural_freqs)


This code reads data from a sensor connected to a Raspberry Pi through an SPI interface, performs FFT analysis on the acquired data to obtain the frequency spectrum of the system's vibrations, and detects the natural frequencies of the system from the peak locations in the spectrum. 